In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import PIL.Image as Image
import random
import numpy as np
import cv2
import time

import time
import sys
sys.path.insert(1,'/home/sclab/Master/I3D-Tensorflow/')
import input_data
import math
from i3d import InceptionI3d
from utils import *
from tensorflow.python import pywrap_tensorflow

In [1]:
import tensorflow as tf

In [ ]:
filename='/home/sclab/Master/I3D-Tensorflow/list/hmdb_list/train_rgb.list'
batch_size=6 
num_frames_per_clip=64
crop_size=224
shuffle=True
gpu_num = 1
num_frame_per_clib = 64
rgb_channels = 3
flow_channels = 2
classics = 8
model_save_dir = '/home/sclab/Master/I3D-Tensorflow/experiments/hmdb-51/models/flow_scratch_20000_6_64_0.0001_decay'

In [ ]:
def get_frames_data(filename, num_frames_per_clip, sample_rate, add_flow):
    ''' Given a directory containing extracted frames, return a video clip of
    (num_frames_per_clip) consecutive frames as a list of np arrays '''
    #print(filename)
    filename_i = os.path.join(filename, 'i')
    #print(filename)
    rgb_ret_arr, s_index = get_data(filename, num_frames_per_clip, sample_rate)
    if not add_flow:
        return rgb_ret_arr, [], s_index
    filename_x = os.path.join(filename, 'x')
    flow_x, _ = get_data(filename_x, num_frames_per_clip, sample_rate, s_index)
    flow_x = np.expand_dims(flow_x, axis=-1)
    filename_y = os.path.join(filename, 'y')
    flow_y, _ = get_data(filename_y, num_frames_per_clip, sample_rate, s_index)
    flow_y = np.expand_dims(flow_y, axis=-1)
    flow_ret_arr = np.concatenate((flow_x, flow_y), axis=-1)
    return rgb_ret_arr, flow_ret_arr, s_index

In [ ]:
def data_process(tmp_data, crop_size):
    img_datas = []
    crop_x = 0
    crop_y = 0
    for j in xrange(len(tmp_data)):
        img = Image.fromarray(tmp_data[j].astype(np.uint8))
        if img.width > img.height:
            scale = float(256) / float(img.height)
            img = np.array(cv2.resize(np.array(img), (int(img.width * scale + 1), 256))).astype(np.float32)
        else:
            scale = float(256) / float(img.width)
            img = np.array(cv2.resize(np.array(img), (256, int(img.height * scale + 1)))).astype(np.float32)
        if j == 0:
            # crop_x = random.randint(0, int(img.shape[0] - crop_size))
            # crop_y = random.randint(0, int(img.shape[1] - crop_size))
            crop_x = int((img.shape[0] - crop_size) / 2)
            crop_y = int((img.shape[1] - crop_size) / 2)
        img = img[crop_x:crop_x + crop_size, crop_y:crop_y + crop_size, :]
        img_datas.append(img)
    return img_datas

In [ ]:
def get_data(filename, num_frames_per_clip, sample_rate, s_index=-1):
#     print (filename)
    ret_arr = []
    filenames = ''
    for parent, dirnames, filenames in os.walk(filename):
        print (len(filenames))
        if len(filenames)==0:
            print('DATA_ERRO: %s'%filename)
            return [], s_index
        if (len(filenames)-s_index) <= num_frames_per_clip:
            filenames = sorted(filenames)
            if len(filenames) < num_frames_per_clip:
                for i in range(num_frames_per_clip):
                    if i >= len(filenames):
                        i = len(filenames)-1
                    image_name = str(filename) + '/' + str(filenames[i])
                    img = Image.open(image_name)
                    img_data = np.array(img)
                    ret_arr.append(img_data)
            else:
                for i in range(num_frames_per_clip):
                    image_name = str(filename) + '/' + str(filenames[len(filenames)-num_frames_per_clip+i])
                    img = Image.open(image_name)
                    img_data = np.array(img)
                    ret_arr.append(img_data)
            return sample_data(ret_arr, num_frames_per_clip, sample_rate), s_index
    filenames = sorted(filenames)
    if s_index < 0:
        s_index = random.randint(0, len(filenames) - num_frames_per_clip)
    for i in range(int(num_frames_per_clip/sample_rate)):
        image_name = str(filename) + '/' + str(filenames[int(i*sample_rate)+s_index])
        img = Image.open(image_name)
        img_data = np.array(img)
        ret_arr.append(img_data)
    return ret_arr, s_index


In [ ]:
lines = open(filename, 'r')
read_dirnames = []
rgb_data = []
flow_data = []
label = []
batch_index = 0
next_batch_start = -1
lines = list(lines)
sample_rate=1
start_pos=-1
add_flow=False
# lines = filename
# Forcing shuffle, if start_pos is not specified
# if start_pos < 0:
shuffle = False
if shuffle:
    video_indices = list(range(len(lines)))
    random.seed(time.time())
    random.shuffle(video_indices)
else:
    # Process videos sequentially
    video_indices = range(start_pos, len(lines))
for index in video_indices:
    if batch_index >= batch_size:
        next_batch_start = index
        break
    line = lines[index].strip('\n').split()
#     print (line)
    dirname = line[0]
    tmp_label = line[1]
    if not shuffle:
        print("Loading a video clip from {}...".format(dirname))
    tmp_rgb_data, tmp_flow_data, _ = get_frames_data(dirname, num_frames_per_clip, sample_rate, add_flow)
    if len(tmp_rgb_data) != 0:
        rgb_img_datas = data_process(tmp_rgb_data, crop_size)
        if add_flow:
            flow_img_datas = data_process(tmp_flow_data, crop_size)
            flow_data.append(flow_img_datas)
        rgb_data.append(rgb_img_datas)
        label.append(int(tmp_label))
        batch_index = batch_index + 1
        read_dirnames.append(dirname)

# pad (duplicate) data/label if less than batch_size
# print (rgb_data)
valid_len = len(rgb_data)
print (valid_len)
pad_len = 12 - valid_len
print (pad_len)
if pad_len:
    for i in range(pad_len):
        rgb_data.append(rgb_data[-1])
#         flow_data.append(flow_data[-1])
        label.append(int(label[-1]))

np_arr_rgb_data = np.array(rgb_data).astype(np.float32)
np_arr_flow_data = np.array(flow_data).astype(np.float32)
np_arr_label = np.array(label).astype(np.int64)

# return np_arr_rgb_data, np_arr_flow_data, np_arr_label.reshape(batch_size), next_batch_start, read_dirnames, valid_len


In [ ]:
len(np_arr_rgb_data[1])

In [ ]:
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)
rgb_pre_model_save_dir = "/home/sclab/Master/I3D-Tensorflow/checkpoints/rgb_imagenet/"

with tf.Graph().as_default():
    global_step = tf.get_variable(
                    'global_step',
                    [],
                    initializer=tf.constant_initializer(0),
                    trainable=False
                    )
    rgb_images_placeholder, flow_images_placeholder, labels_placeholder, is_training = placeholder_inputs(
                    batch_size * gpu_num,
                    num_frame_per_clib,
                    crop_size,
                    rgb_channels,
                    flow_channels
                    )

    learning_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps=3000, decay_rate=0.1, staircase=True)
    opt_rgb = tf.train.AdamOptimizer(learning_rate)
    #opt_stable = tf.train.MomentumOptimizer(learning_rate, 0.9)
    with tf.variable_scope('RGB'):
        rgb_logit, _ = InceptionI3d(
                                num_classes=classics,
                                spatial_squeeze=True,
                                final_endpoint='Logits'
                                )(rgb_images_placeholder, is_training)
    rgb_loss = tower_loss(
                            rgb_logit,
                            labels_placeholder
                            )
    accuracy = tower_acc(rgb_logit, labels_placeholder)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        rgb_grads = opt_rgb.compute_gradients(rgb_loss)
        apply_gradient_rgb = opt_rgb.apply_gradients(rgb_grads, global_step=global_step)
        train_op = tf.group(apply_gradient_rgb)
        null_op = tf.no_op()

    # Create a saver for loading trained checkpoints.
    rgb_variable_map = {}
    for variable in tf.global_variables():
        if variable.name.split('/')[0] == 'RGB' and 'Adam' not in variable.name.split('/')[-1] and variable.name.split('/')[2] != 'Logits':
            # rgb_variable_map[variable.name.replace(':0', '')[len('RGB/inception_i3d/'):]] = variable
            rgb_variable_map[variable.name.replace(':0', '')] = variable
    rgb_saver = tf.train.Saver(var_list=rgb_variable_map, reshape=True)

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()
    init = tf.global_variables_initializer()

    # Create a session for running Ops on the Graph.
    sess = tf.Session(
                    config=tf.ConfigProto(allow_soft_placement=True)
                    )
    sess.run(init)
    # Create summary writter
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('rgb_loss', rgb_loss)
    tf.summary.scalar('learning_rate', learning_rate)
    merged = tf.summary.merge_all()
# load pre_train models
ckpt = tf.train.get_checkpoint_state(rgb_pre_model_save_dir)
print (ckpt)
if ckpt and ckpt.model_checkpoint_path:
    print("loading checkpoint %s,waiting......" % ckpt.model_checkpoint_path)
    rgb_saver.restore(sess, ckpt.model_checkpoint_path)
    print("load complete!")

train_writer = tf.summary.FileWriter('/home/sclab/Master/I3D-Tensorflow/visual_logs/train_rgb_imagenet_10000_6_64_0.0001_decay', sess.graph)
test_writer = tf.summary.FileWriter('/home/sclab/Master/I3D-Tensorflow/visual_logs/test_rgb_imagenet_10000_6_64_0.0001_decay', sess.graph)
for step in xrange(max_steps):
    start_time = time.time()
#         rgb_train_images, flow_train_images, train_labels, _, _, _ = input_data.read_clip_and_label(
#                       filename='/home/sclab/Master/I3D-Tensorflow/list/hmdb_list/train_rgb.list',
#                       batch_size=FLAGS.batch_size * gpu_num,
#                       num_frames_per_clip=FLAGS.num_frame_per_clib,
#                       crop_size=FLAGS.crop_size,
#                       shuffle=True
#                       )
    # print (rgb_train_images)
    # print (rgb_train_images.shape())
    sess.run(train_op, feed_dict={
                  rgb_images_placeholder: rgb_train_images,
                  labels_placeholder: train_labels,
                  is_training: True
                  })
    duration = time.time() - start_time
    print('Step %d: %.3f sec' % (step, duration))

    # Save a checkpoint and evaluate the model periodically.
    if step % 10 == 0 or (step + 1) == FLAGS.max_steps:
        print('Training Data Eval:')
        summary, acc, loss_rgb = sess.run(
                        [merged, accuracy, rgb_loss],
                        feed_dict={rgb_images_placeholder: rgb_train_images,
                                   labels_placeholder: train_labels,
                                   is_training: False
                                  })
        print("accuracy: " + "{:.5f}".format(acc))
        print("rgb_loss: " + "{:.5f}".format(loss_rgb))
        train_writer.add_summary(summary, step)
        print('Validation Data Eval:')
        rgb_val_images, flow_val_images, val_labels, _, _, _ = input_data.read_clip_and_label(
                        filename='/home/sclab/Master/I3D-Tensorflow/list/hmdb_list/test_flow.list',
                        batch_size=batch_size * gpu_num,
                        num_frames_per_clip=num_frame_per_clib,
                        crop_size=crop_size,
                        shuffle=True
                        )
        summary, acc = sess.run(
                        [merged, accuracy],
                        feed_dict={
                                    rgb_images_placeholder: rgb_val_images,
                                    labels_placeholder: val_labels,
                                    is_training: False
                                  })
        print("accuracy: " + "{:.5f}".format(acc))
        test_writer.add_summary(summary, step)
    if (step+1) % 3000 == 0 or (step + 1) == FLAGS.max_steps:
        saver.save(sess, os.path.join(model_save_dir, 'i3d_hmdb_model'), global_step=step)
print("done")